In [58]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Layer Norm by hand
- Fill in the Layer Norm application below. Make sure the manual and PyTorch implementations are the same.
- As before, FILL_IN the missing code to make this work.

In [59]:
FILL_IN = "FILL_IN"

In [191]:
# This is (N, T, d_model)
# N: batch size
# T: sentence_length
# d_model: embedding dimension

N, T, d_model = 2, 3, 4

# An embedding. This is what you might feed into a network.
embedding = torch.randn(N, T, d_model)

# Create a Layer Norm layer on the embedding dimension.
# Do not include gamma and beta, the learnable scaling and offset parameters.
# This should act of the dimenson of the model.
layer_norm = FILL_IN
# Run embedding through the layer_norm layer.
layer_norm_pytorch = FILL_IN

# Manual computation; use the same EPSILON as is used in the standard nn.LayerNorm.
EPSILON = FILL_IN

# Grab the mean of each vector in the first batch. This should be (3, 1).
mean = FILL_IN
# Grab the var of each vector in the first batch. This should be (3, 1).
var = FILL_IN
# Manually take each vector in the batch and standerdize it.
layer_norm_manual = FILL_IN
print("layer_norm_manual @ 0: ", layer_norm_manual)
print("layer_norm_out[0]: ", layer_norm_pytorch[0])
assert torch.allclose(layer_norm_pytorch[0], layer_norm_manual), 'Tensors do not match.'

mean = FILL_IN
var = FILL_IN
layer_norm_manual = FILL_IN
print("layer_norm_manual @ 1: ", layer_norm_manual)
print("layer_norm_out[1]: ", layer_norm_pytorch[1])
assert torch.allclose(layer_norm_pytorch[1], layer_norm_manual), 'Tensors do not match.'

layer_norm_manual @ 0:  tensor([[ 1.5415, -0.3528, -1.2239,  0.0352],
        [-1.0197, -0.9560,  1.2051,  0.7707],
        [ 0.2628,  1.2335, -1.5512,  0.0549]])
layer_norm_out[0]:  tensor([[ 1.5415, -0.3528, -1.2239,  0.0352],
        [-1.0197, -0.9560,  1.2051,  0.7707],
        [ 0.2628,  1.2335, -1.5512,  0.0549]])
layer_norm_manual @ 1:  tensor([[ 1.4899, -1.1755,  0.2604, -0.5748],
        [-1.6177,  0.3001,  0.1980,  1.1197],
        [ 1.0943,  0.9008, -1.0170, -0.9781]])
layer_norm_out[1]:  tensor([[ 1.4899, -1.1755,  0.2604, -0.5748],
        [-1.6177,  0.3001,  0.1980,  1.1197],
        [ 1.0943,  0.9008, -1.0170, -0.9781]])


### Wave Net

In [207]:
# Hyperparameters we will use.                                                                                                                                                          
batch_size = 128 # How many independent sequences will we process in parallel?                                                                                              
context_size = 256 # What is the maximum context length for predictions? This is T below.                                                                                                    
epochs = 5000
eval_interval = 500
# Is this a good one? Can you check?
learning_rate = 3e-4
device = 'cpu' # Do this if you have a MAC: 'mps' if torch.backends.mps.is_available() else 'cpu'
eval_iters = 200
d_model = 20
d_hidden = 100
n_layer = 1
dropout = 0.2
write_to_file = False
norm = 'batch_norm'

# Add more pritning to the model.
debug = False
# ------------        

In [208]:
torch.manual_seed(1337)

# Load the Shakespere document input.txt.                                                                          
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [209]:
# Get all the unique characters in the text.                                                                                                             
chars = FILL_IN
vocab_size = FILL_IN
# As usual, create a mapping from a character to a text.                                                                                                                            
stoi = FILL_IN
itos = FILL_IN
# encode: is a lambda function that takes a string and returns  a list of ints, where each character is mapped to the right int.
encode = FILL_IN
# decode: is the reverse mapping of encode. It takes a list of int, and returns a string.
decode = FILL_IN   

In [210]:
# Do a train-test split with 90% of the data train and 10% test.
# You can just use the first 90% of the data as training data.
# Run the text through the encode method.
data = FILL_IN
n = int(0.9*len(data)) # first 90% will be train, rest val                                                                                                                 
train_data = FILL_IN
val_data = FILL_IN


In [212]:
# Load the data.
# This should return a small batch of data (x, y) where x is 
def get_batch(split):
    # Generate a small batch of data of inputs x and targets y.
    # Pick the train data if split == 'train', else the validation data.
    data = FILL_IN
    # Select a random set of ints [0, len(data) - context_size) ; reshape this to be (batch_size, )
    # For an index i, a x should be data[i:i+context_size] while a y should be data[i+context_size].
    # ix has length batch_size.
    ix = FILL_IN
    # Stack the batch_size data to be of shape (batch_size, context_size)
    x = FILL_IN
    # Stack the y targets; this should be of length batch_size.
    # You should pull out the i+context_size element of data; i is an index in ix.
    y = FILL_IN                                                                                                                 
    x, y = x.to(device), y.to(device)
    return x, y

In [213]:
# Estimate the loss.

@torch.no_grad()
def estimate_loss():
    out = {}
    # Put the model in eval mode. Why?
    FILL_IN
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            logits, loss = model(xb, yb)
            # Get the value in the loss.
            losses[k] = FILL_IN
        # Get the mean of the values in the losses.
        out[split] = FILL_IN
    # Put the model in train mode.
    FILL_IN
    return out

In [220]:
class WaveNetMLPLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
        self.token_embedding_table = nn.Embedding(vocab_size, d_model)
        
        self.linear_layers = []
        self.norm_layers = []
        
        temp_context_size = context_size
        
        while temp_context_size >= 10:
            # Map from 2 * d_model to d_hidden.
            if not self.linear_layers:
                # Add to linear_layers a layer going 2 * d_model to d_hidden.
                FILL_IN
            else:
                # Map from 2 * d_hidden to d_hidden.
                # Add to linear_layers a layer going 2 * d_hidden to d_hidden.
                FILL_IN
            # Append to norm_layers a batch norm 1d with vectors of size d_hidden.
            FILL_IN
            
            temp_context_size //= 2
        
        # Add a final batch norm 1d with vectors of size vocab_size. 
        self.norm_f = FILL_IN # Final layer norm.
        # Add a Linear layer going from temp_context_size * d_hidden to vocab_size.
        self.ff = FILL_IN
        
    def forward(self, idx, targets=None):
        N, T = idx.shape

        # idx and targets are both (N, T) tensor of integers                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
        x = self.token_embedding_table(idx) # (N, T, d_model)
        
        for i, _ in enumerate(self.linear_layers):
            N, T, D = x.shape
            # Reshape x to be (N, ??). You want to shrink the context window down by two each time.
            x = FILL_IN
            # Pass through linear layer i.
            x = FILL_IN
            # Transpose appropriate dimensions of x. Look at the expected dimensions of BatchNorm1d.
            x = FILL_IN
            # Pass through the batch norm layer.
            x = FILL_IN
            # Transpose back to the previous dimensions.
            x = FILL_IN
            # Pass through ReLU.
            x = FILL_IN
        
        # Reshape.
        x = FILL_IN # (N, [T // (2 ** len(self.linear_layers))] * d_model)
        
        # Apply dropout.
        x = FILL_IN
        
        # Apply self.ff.
        x = FILL_IN # (N, vocab_size)
        
        # Apply batch norm.
        x = FILL_IN

        # Apply Tanh.
        logits = FILL_IN

        if targets is None:
            loss = None
        else:
            _, T = logits.shape

            assert(T == vocab_size)

            # Apply cross entropy.
            loss = FILL_IN

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is a (N, T) array of indices in the current context.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
        for _ in range(max_new_tokens):
            # Here, we crop idx to the last context_size tokens.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
            idx_cond = FILL_IN
            # Get the predictions; this is just the last timestep.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
            logits, loss = FILL_IN
            # Apply softmax to get probabilities.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
            probs = FILL_IN # (N, vocab_size)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
            # Sample from the distribution to get the next character's index.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
            idx_next = FILL_IN # (N, 1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
            # Append sampled index to the running sequence.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
            idx = FILL_IN # (N, T+1)
        return idx # At most, this is (N, T + max_new_tokens) in the second dimension.

Train the model.

In [221]:
model = WaveNetMLPLanguageModel().to(device)
# Print the number of parameters in the model.
print(FILL_IN, 'M parameters')

# Create a PyTorch optimizer. Use AdamW.                                                                                                                                                                                                                                        
optimizer = FILL_IN

0.331695 M parameters


In [2]:
# Here we loop over max_iters and at each iter we get a batch of data we optimize over.
model.train()
for epoch in range(epochs):

    # every once in a while evaluate the loss on train and val sets                                                                                                                                                                                                 
    if epoch % eval_interval == 0 or epoch == epochs - 1:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data                                                                                                                                                                                                                                        
    xb, yb = get_batch('train')

    # evaluate the loss                                                                                                                                                                                                                                             
    logits, loss = model(xb, yb)
    # Zero the grads.
    FILL_IN
    # Get gradients by backprop; do a parameter update.
    FILL_IN
    FILL_IN

In [ ]:
# Generate from the model and save it to wave_net.txt.
# We generate a maximum of 1000 tokens. 
# We feed in a batch of dimenson (1, context_size).
# The loss should get to ~ 2.0 on train and validation.
# Unfortunately, this will likely not make much sense, the capacity of this model is not ideal for this task.
# The name generation task fro HW 1 might be aother data set to use.
model.eval()
context = train_data[:256].reshape(1, 256)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))
if write_to_file:
    open('wave_net.txt', 'w').write(decode(model.generate(context, max_new_tokens=10000)[0].tolist()))

Bonus (+5 max - If you do this and it's all right this assignment will be 13/10.)
- Add some residual connections. Does this improve gradient zero issues?
 - Add some logging to figure out the number of zero gradients across the network before and after you add the residual connections.
- Add some plots that show the train and validation loss, per k iterations. You might want k < 500.
- Use LayerNorm instead of batch norm.
- Use the names.txt file from assignment 1. How do the names look?